# **1. Kết nối với google drive**

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/NLP 4+1/BTL3- BERT and GPT/')
os.getcwd()

Mounted at /content/drive


'/content/drive/.shortcut-targets-by-id/1bs2wermM-w1hKEOcHyv-UmLHY5iag5cm/NLP 4+1/BTL3- BERT and GPT'

# **2. Install và import các thư viện cần thiết**

In [ ]:
# install library
!pip install transformers --quiet
!pip install keras_preprocessing --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.5 MB/s eta 0:00:00


In [ ]:
import torch
import time
import random
import datetime
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import get_linear_schedule_with_warmup
from transformers import BertTokenizer, BertLMHeadModel, AdamW, BertConfig, BertModel


# **3. Set các biến cục bộ và load dữ liệu**

In [ ]:
import pickle

# saving
def save_file(data, path):
    with open(path + '.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
def load_file(path):
    with open(path, 'rb') as handle:
        data = pickle.load(handle)
    return data

In [ ]:
# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
EPOCHS = 100
BATCH_SIZE = 32
path_data = './dataset/'                    # đường dẫn data
path_pre_trained = './pre-trained/BERT/'    # đường dẫn chứa các model đã train

In [ ]:
# load data
df_train = pd.read_csv(path_data + 'train.csv')
df_train

,question,answer
0,what canbe the cause for chlamydia not going a...,i do not know how the chlamydia was treated wa...
1,can you overfeed an infant,i really do not have enough information to spe...
2,can you catch a cold through oral sex,hi i think your find its a hard thing to do un...
3,why do i get ring worm ever year ever year i r...,it is not a worm ringworm is just the lay term...
4,what ingredients should i look for in toothpaste,use fluoride toothpaste to help avoid cavities...
...,...,...
6260,is spinal meningitis contagious,yes some types of meningitis are contagious so...
6261,can exercise delay a period 20 days late past ...,hi women who exercise do tend to get skipped p...
6262,how do i care for my dental implant,dental implants require the same care as real ...
6263,can you get pregnant through underwear me and ...,no you have to have sex to get pregnant be car...


# **4. Tiền xử lý dữ liệu**

## ***4.1 Convert string to ids***

In [ ]:
# Load the BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained('cambridgeltl/BioRedditBERT-uncased', do_lower_case=True)

# save
save_file(tokenizer, path_pre_trained + 'tokenizer')

In [ ]:
# Get the lists of questions and their answers.
questions = df_train['question'].values
answers = df_train['answer'].values

In [ ]:
# Tokenize all of the questions and map the tokens to thier word IDs.
questions_ids = []
answers_ids = []

for sent, lb in zip(questions, answers):
    encoded_sent = tokenizer.encode(sent, add_special_tokens = True)
    encoded_lb = tokenizer.encode(lb, add_special_tokens = True, )
    
    # Add the encoded sentence to the list.
    questions_ids.append(encoded_sent)
    answers_ids.append(encoded_lb)

In [ ]:
# Print sentence 0, now as a list of IDs.
print('Original: ', questions[0])
print('Token IDs:', questions_ids[0])
print('Decode oken IDs:', tokenizer.decode(questions_ids[0]))

print()

print('Original: ', answers[0])
print('Token IDs:', answers_ids[0])
print('Decode oken IDs:', tokenizer.decode(answers_ids[0]))

Original:  what canbe the cause for chlamydia not going away after treatment and no sexual intercourse after being diagnosed
Token IDs: [101, 1184, 1169, 3962, 1103, 2612, 1111, 22572, 7609, 19429, 1465, 1136, 1280, 1283, 1170, 3252, 1105, 1185, 3785, 26864, 1170, 1217, 11534, 102]
Decode oken IDs: [CLS] what canbe the cause for chlamydia not going away after treatment and no sexual intercourse after being diagnosed [SEP]

Original:  i do not know how the chlamydia was treated was the dose appropriate and taken long enough were you compliant in taking it properly are you male or female do have more than one partner as much as i would like to address your question i really do not have enough information
Token IDs: [101, 178, 1202, 1136, 1221, 1293, 1103, 22572, 7609, 19429, 1465, 1108, 5165, 1108, 1103, 13753, 5806, 1105, 1678, 1263, 1536, 1127, 1128, 3254, 27898, 1107, 1781, 1122, 7513, 1132, 1128, 2581, 1137, 2130, 1202, 1138, 1167, 1190, 1141, 3547, 1112, 1277, 1112, 178, 1156, 1176,

In [ ]:
print(tokenizer.convert_ids_to_tokens([1169,3962]))

['can', '##be']


## ***4.2 Padding & Truncating***

In [ ]:
print('Max sentence length: ', max([len(sen) for sen in questions_ids]))
print('Max labels length: ', max([len(lb) for lb in answers_ids]))

Max sentence length:  77
Max labels length:  120


In [ ]:
# Pad and truncate our sequences so that they all have the same length, MAX_LEN.
MAX_LEN = 120
questions_ids = pad_sequences(questions_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
answers_ids = pad_sequences(answers_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
questions_ids[0]

array([  101,  1184,  1169,  3962,  1103,  2612,  1111, 22572,  7609,
       19429,  1465,  1136,  1280,  1283,  1170,  3252,  1105,  1185,
        3785, 26864,  1170,  1217, 11534,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0])

## ***4.3 Attention Masks***

In [ ]:
# Create attention masks
attention_masks = []

for ques in questions_ids:
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask_sent = [int(token_id > 0) for token_id in ques]
    
    # Store the attention mask for this questions.
    attention_masks.append(att_mask_sent)

print(attention_masks[0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## ***4.4 Converting to PyTorch Data Types***

In [ ]:
train_inputs = torch.tensor(questions_ids)
train_labels = torch.tensor(answers_ids)
train_masks = torch.tensor(attention_masks)

In [ ]:
# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

# **5. Build model chatbot with Classification Model**

## ***5.1 Load model pre-trained***

In [ ]:
# Load BertForSequenceClassification, the pretrained BERT model with a single  linear classification layer on top. 
model = BertLMHeadModel.from_pretrained("cambridgeltl/BioRedditBERT-uncased")

# Tell pytorch to run this model on the GPU.
model = model.to(device)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at cambridgeltl/BioRedditBERT-uncased and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## ***5.2 Optimizer & Learning Rate Scheduler***

In [ ]:
optimizer = AdamW(model.parameters(),lr = 2e-5,eps = 1e-8)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

## ***5.3 Training Loop***

### 5.3.1 Hàm trả về câu trả lời của chatbot

In [ ]:
def bot_give_answer(inp):
    inputs = tokenizer(inp, max_length=MAX_LEN, padding='max_length', return_tensors="pt").to(device)
    outputs = model(**inputs, labels=inputs["input_ids"])

    result = []
    for word_embeds in outputs.logits[0]:
        word_embed = word_embeds
        # find one shot index from word embedding
        max_idx_t = word_embed.argmax()
        max_idx = max_idx_t.item()
        word = tokenizer.convert_ids_to_tokens(max_idx)
        result.append(word)
    return ' '.join(result)

### 5.3.2 Training

In [ ]:
# Store the average loss after each epoch so we can plot them.
loss_history = []
t0 = time.time()
for epoch_i in range(0, EPOCHS):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, EPOCHS))
    total_loss = 0
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    torch.save(model.state_dict(), path_pre_trained + "model_BERT.pt")
    print('Question: ', 'do i need to exercise to benefit from vitamin c')
    print('Answer:   ', bot_give_answer('do i need to exercise to benefit from vitamin c'))

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_history.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
time_train = time.time() - t0

======== Epoch 1 / 100 ========


  0%|          | 0/196 [00:00<?, ?it/s]

Question:  do i need to exercise to benefit from vitamin c
Answer:    [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

  Average training loss: 3.26
======== Epoch 2 / 100 ========


  0%|          | 0/196 [00:00<?, ?it/s]

Question:  do i need to exercise to benefit from vitamin c
Answer:    no you you you you you you you you you [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

  Average training loss: 2.65
======== Epoch 3 / 100 ========


...

Question:  do i need to exercise to benefit from vitamin c
Answer:    no vitamin c is not to exercise but absorption not exercise no [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

  Average training loss: 1.41
======== Epoch 99 / 100 ========


  0%|          | 0/196 [00:00<?, ?it/s]

Question:  do i need to exercise to benefit from vitamin c
Answer:    no vitamin c is exercise without exercise but absorption not not no [PAD] exercise [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

  Average training loss: 1.41
======== Epoch 100 / 100 ========


  0%|          | 0/196 [00:00<?, ?it/s]

Question:  do i need to exercise to benefit from vitamin c
Answer:    no vitamin c is not to exercise but do not not no not [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

  Average training loss: 1.41


In [ ]:
val_compare = {
    'loss_history': loss_history,
    'time_train': time_train
}

# save
save_file(val_compare, path_pre_trained + 'val_compare')